In [1]:
import fiona
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import geopandas as gpd
from shapely.geometry import Point, Polygon

import zipfile
import requests
import os
import shutil

from sklearn.preprocessing import *

from lnks import scl_cols

import tensorflow as tf

%matplotlib inline

# To start...

We import the data output of our data pipeline. We reset the index, drop index columns, and lag the data.

We explicitly print shape several times, making sure that we capture the magnitude of data lost from dropping NA values.

In [2]:
#Get data
filestring = './data/anc_out.csv'
df = pd.read_csv(filestring)

df = df.reset_index()

print(df.shape)
df['y']= df['countBBL'].shift(-40)

df = df.dropna()
df = df.drop('index_left', axis= 1)
print(df.shape)
df = df.fillna(0)
df = df.astype('float')
df = df[df.month < df.month.max()]
df = df.dropna()
print(df.shape)

(2400, 47)
(2360, 47)
(2320, 47)


In [3]:
df[['month', 'countBBL', 'y']][df['NAME_ANC 1D']==1]

,month,countBBL,y
1,2012.01,66.0,63.0
41,2012.02,63.0,66.0
81,2012.03,66.0,66.0
121,2012.04,66.0,72.0
161,2012.05,72.0,72.0
201,2012.06,72.0,75.0
241,2012.07,75.0,78.0
281,2012.08,78.0,80.0
321,2012.09,80.0,87.0
361,2012.10,87.0,93.0


In [4]:
df.dtypes

index                float64
Util_Indx_BBL        float64
countBBL             float64
countIssued          float64
month                float64
BIZ_Dist_Concentr    float64
NAME_ANC 1A          float64
NAME_ANC 1B          float64
NAME_ANC 1C          float64
NAME_ANC 1D          float64
NAME_ANC 2A          float64
NAME_ANC 2B          float64
NAME_ANC 2C          float64
NAME_ANC 2D          float64
NAME_ANC 2E          float64
NAME_ANC 2F          float64
NAME_ANC 3B          float64
NAME_ANC 3C          float64
NAME_ANC 3D          float64
NAME_ANC 3E          float64
NAME_ANC 3F          float64
NAME_ANC 3G          float64
NAME_ANC 4A          float64
NAME_ANC 4B          float64
NAME_ANC 4C          float64
NAME_ANC 4D          float64
NAME_ANC 5A          float64
NAME_ANC 5B          float64
NAME_ANC 5C          float64
NAME_ANC 5D          float64
NAME_ANC 5E          float64
NAME_ANC 6A          float64
NAME_ANC 6B          float64
NAME_ANC 6C          float64
NAME_ANC 6D   

In [9]:
#Flexible adaptation of Dr. Braman's interactive gridsearch script
#implementation. 
#TODO Clean up and streamline
import sklearn
from sklearn.neural_network import *
from sklearn.neighbors import *
from sklearn.svm import *
from sklearn.gaussian_process import *
from sklearn.gaussian_process.kernels import *
from sklearn.tree import *
from sklearn.ensemble import *
from sklearn.naive_bayes import *
from sklearn.discriminant_analysis import *
from sklearn.linear_model import *
from sklearn.model_selection import *
from sklearn.preprocessing import *
import random

data = df #copy it

#Frame up some separate DataFrames for scalar and stuff
scl_data = data




data     = data.reset_index()
data.drop(['level_0', 'index'], axis=1, inplace=True)
X = data.drop(['y'], axis=1)
y = data['y']
ytr = sklearn.preprocessing.MinMaxScaler([0, 1]
            ).fit(y)
y = ytr.fit_transform(y)

scl_data = scl_data.reset_index()
                            
scl_data = scl_data.drop(['level_0', 'index'
                         ], axis=1)

/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:356: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.re

In [10]:
#Build scalers for the scl_data, other --------------------

scaler = sklearn.preprocessing.StandardScaler(
            ).fit(scl_data)
minmaxer = sklearn.preprocessing.MinMaxScaler([0, 1]
            ).fit(scl_data)
scaler
#scl_data[scl_data.columns
#   ] = scaler.fit_transform(scl_data[scl_data.columns])

#----------------------------------------------------------


StandardScaler(copy=True, with_mean=True, with_std=True)

In [11]:
scl = scaler.transform(scl_data)
scl = minmaxer.transform(scl_data)
scl_data = pd.DataFrame(scl, columns=scl_data.columns)

In [12]:
print(scl_data.shape)
scl_data = scl_data.dropna()
scl_data

(2320, 46)


,Util_Indx_BBL,countBBL,countIssued,month,BIZ_Dist_Concentr,NAME_ANC 1A,NAME_ANC 1B,NAME_ANC 1C,NAME_ANC 1D,NAME_ANC 2A,...,NAME_ANC 7C,NAME_ANC 7D,NAME_ANC 7E,NAME_ANC 7F,NAME_ANC 8A,NAME_ANC 8B,NAME_ANC 8C,NAME_ANC 8D,NAME_ANC 8E,y
0,0.140082,0.060843,0.003086,0.0,0.998646,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.068056
1,0.062919,0.015296,0.000000,0.0,0.264922,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.013676
2,0.186277,0.072740,0.009259,0.0,0.800974,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.069684
3,0.226157,0.247111,0.021605,0.0,0.891499,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.239336
4,0.311580,0.135962,0.003086,0.0,1.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.132530
5,0.045850,0.000000,0.000000,0.0,0.253874,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000326
6,0.057269,0.053365,0.024691,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.052426
7,0.119139,0.095173,0.009259,0.0,0.792841,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.089222
8,0.051567,0.032291,0.012346,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.029958
9,0.050770,0.061863,0.003086,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.060567


In [13]:
scl_data.columns

Index(['Util_Indx_BBL', 'countBBL', 'countIssued', 'month',
       'BIZ_Dist_Concentr', 'NAME_ANC 1A', 'NAME_ANC 1B', 'NAME_ANC 1C',
       'NAME_ANC 1D', 'NAME_ANC 2A', 'NAME_ANC 2B', 'NAME_ANC 2C',
       'NAME_ANC 2D', 'NAME_ANC 2E', 'NAME_ANC 2F', 'NAME_ANC 3B',
       'NAME_ANC 3C', 'NAME_ANC 3D', 'NAME_ANC 3E', 'NAME_ANC 3F',
       'NAME_ANC 3G', 'NAME_ANC 4A', 'NAME_ANC 4B', 'NAME_ANC 4C',
       'NAME_ANC 4D', 'NAME_ANC 5A', 'NAME_ANC 5B', 'NAME_ANC 5C',
       'NAME_ANC 5D', 'NAME_ANC 5E', 'NAME_ANC 6A', 'NAME_ANC 6B',
       'NAME_ANC 6C', 'NAME_ANC 6D', 'NAME_ANC 6E', 'NAME_ANC 7B',
       'NAME_ANC 7C', 'NAME_ANC 7D', 'NAME_ANC 7E', 'NAME_ANC 7F',
       'NAME_ANC 8A', 'NAME_ANC 8B', 'NAME_ANC 8C', 'NAME_ANC 8D',
       'NAME_ANC 8E', 'y'],
      dtype='object')

In [14]:
sX = scl_data.drop(['y'], axis=1)
sy = scl_data['y']



assert np.all(np.isfinite(X))
assert np.all(np.isfinite(y))
assert not np.any(np.isnan(X))
assert not np.any(np.isnan(y))

assert np.all(np.isfinite(sX))
assert np.all(np.isfinite(sy))
assert not np.any(np.isnan(sX))
assert not np.any(np.isnan(sy))

In [15]:
scl_data.columns

Index(['Util_Indx_BBL', 'countBBL', 'countIssued', 'month',
       'BIZ_Dist_Concentr', 'NAME_ANC 1A', 'NAME_ANC 1B', 'NAME_ANC 1C',
       'NAME_ANC 1D', 'NAME_ANC 2A', 'NAME_ANC 2B', 'NAME_ANC 2C',
       'NAME_ANC 2D', 'NAME_ANC 2E', 'NAME_ANC 2F', 'NAME_ANC 3B',
       'NAME_ANC 3C', 'NAME_ANC 3D', 'NAME_ANC 3E', 'NAME_ANC 3F',
       'NAME_ANC 3G', 'NAME_ANC 4A', 'NAME_ANC 4B', 'NAME_ANC 4C',
       'NAME_ANC 4D', 'NAME_ANC 5A', 'NAME_ANC 5B', 'NAME_ANC 5C',
       'NAME_ANC 5D', 'NAME_ANC 5E', 'NAME_ANC 6A', 'NAME_ANC 6B',
       'NAME_ANC 6C', 'NAME_ANC 6D', 'NAME_ANC 6E', 'NAME_ANC 7B',
       'NAME_ANC 7C', 'NAME_ANC 7D', 'NAME_ANC 7E', 'NAME_ANC 7F',
       'NAME_ANC 8A', 'NAME_ANC 8B', 'NAME_ANC 8C', 'NAME_ANC 8D',
       'NAME_ANC 8E', 'y'],
      dtype='object')

In [16]:
scl_data.describe()

,Util_Indx_BBL,countBBL,countIssued,month,BIZ_Dist_Concentr,NAME_ANC 1A,NAME_ANC 1B,NAME_ANC 1C,NAME_ANC 1D,NAME_ANC 2A,...,NAME_ANC 7C,NAME_ANC 7D,NAME_ANC 7E,NAME_ANC 7F,NAME_ANC 8A,NAME_ANC 8B,NAME_ANC 8C,NAME_ANC 8D,NAME_ANC 8E,y
count,2320.000000,2320.000000,2320.000000,2320.000000,2320.000000,2320.000000,2320.000000,2320.000000,2320.000000,2320.000000,...,2320.000000,2320.000000,2320.000000,2320.000000,2320.000000,2320.000000,2320.000000,2320.000000,2320.000000,2320.000000
mean,0.189663,0.107119,0.060873,0.485161,0.270316,0.025000,0.025000,0.025000,0.025000,0.025000,...,0.025000,0.025000,0.025000,0.025000,0.025000,0.025000,0.025000,0.025000,0.025000,0.106209
std,0.149830,0.094190,0.092099,0.338886,0.379395,0.156159,0.156159,0.156159,0.156159,0.156159,...,0.156159,0.156159,0.156159,0.156159,0.156159,0.156159,0.156159,0.156159,0.156159,0.094574
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.089450,0.046227,0.009259,0.249389,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.045262
50%,0.145863,0.082937,0.027778,0.500000,0.014064,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.081570
75%,0.243307,0.139021,0.074074,0.750611,0.704218,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.137415
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [17]:
#Make a short list of random states to insert into randomstate params.
scrambler = []
for scram in range(0, 10):
    scrambler.append(random.randint(0, 10000))   
print(scrambler)

to_scale = ['SVR']

names       = ['AdaBoostRegressor',
             'RandomForestRegressor',
             'SVR',
             #'KNeighborsRegressor',
             #'BaggingRegressor',
             'GradientBoostingRegressor',
             #'LinearRegression',
             #'MLPRegressor',
             #'SGDRegressor',
             'LassoLars'         
    
]

regressors = [AdaBoostRegressor(),
              RandomForestRegressor(),
              SVR(),
              #KNeighborsRegressor(),
              #BaggingRegressor(),
              GradientBoostingRegressor(),
              #LinearRegression(),
              #MLPRegressor(),
              #SGDRegressor(),
              LassoLars()
    
]

param_grids =[ 
    ['AdaBoostRegressor', dict(
        n_estimators=[80, 60, 30],
        learning_rate=[1, .5, .01],
        loss=['linear', 'square', 'exponential'],
        random_state=scrambler[3:5]
        
    )],
        
    ['RandomForestRegressor', dict(
        max_depth=[5, 10],
        #criterion=['mse', 'mae'],
        random_state=scrambler[:2]
    )],
    ['SVR', dict(
        C=[1, .75, .5],
        epsilon=[.1, .05],
        kernel=['rbf', 'sigmoid', 'poly']
    )],
    ['GradientBoostingRegressor', dict(
        max_depth=[3, 6, 9],
        min_samples_split=[2, 4, 8],
        random_state=scrambler[5:8],
        presort=['auto', False]
    )],
    ['LassoLars', dict(
        alpha=[0.1, 1, .5, .75],
        fit_path=[True, False]
    )],
    ]

[669, 4710, 2654, 865, 3802, 6677, 4103, 2382, 7351, 2623]


In [18]:
outcomes = []
for name, rgsr in zip(names, regressors):
    
    for item in param_grids:
        if item[0]==name:
            params= item[1]
        
    
    cv = sklearn.model_selection.GridSearchCV(rgsr, param_grid=params,
                                              verbose=True, n_jobs=12,
                                              cv=5, pre_dispatch="2*n_jobs")
    X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(
        X, y, random_state=random.randint(0, 10000))
    
    #Overwrite to scalar data model is on list of
    #scalar-data-optimized algos
    if name in to_scale:
        X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(
            sX, sy, random_state=random.randint(0, 10000))
    
    fitted = cv.fit(X_train, y_train)
    score = cv.score(X_test, y_test)
    outcomes.append((name, score, cv.cv_results_, cv.best_estimator_))
    print(name, ' is done!')
    print(name, ' scored:', score)
    
for nm in range(0, len(outcomes)):
    print()
    print(outcomes[nm][0])
    print(outcomes[nm][1])
    print()
    
    

Fitting 5 folds for each of 54 candidates, totalling 270 fits


[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    8.4s
[Parallel(n_jobs=12)]: Done 176 tasks      | elapsed:   53.1s
[Parallel(n_jobs=12)]: Done 270 out of 270 | elapsed:  1.5min finished


AdaBoostRegressor  is done!
AdaBoostRegressor  scored: 0.988272354828
Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=12)]: Done  18 out of  20 | elapsed:    3.7s remaining:    0.4s
[Parallel(n_jobs=12)]: Done  20 out of  20 | elapsed:    3.7s finished


RandomForestRegressor  is done!
RandomForestRegressor  scored: 0.996234775259
Fitting 5 folds for each of 18 candidates, totalling 90 fits


...

SVR  is done!
SVR  scored: 0.933386909773
Fitting 5 folds for each of 54 candidates, totalling 270 fits


[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    8.8s
[Parallel(n_jobs=12)]: Done 176 tasks      | elapsed:  1.2min
[Parallel(n_jobs=12)]: Done 270 out of 270 | elapsed:  2.5min finished


GradientBoostingRegressor  is done!
GradientBoostingRegressor  scored: 0.996343351012
Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=12)]: Done  40 out of  40 | elapsed:    4.9s finished


LassoLars  is done!
LassoLars  scored: -0.000116758107285

AdaBoostRegressor
0.988272354828


RandomForestRegressor
0.996234775259


SVR
0.933386909773


GradientBoostingRegressor
0.996343351012


LassoLars
-0.000116758107285



In [19]:
dat16 = data[data['month'] > 2015.5]
dat15 = data[data['month'] < 2015.5]

X16 = dat16.drop(['y'], axis=1)
y16 = dat16['y']
X15 = dat15.drop(['y'], axis=1)
y15 = dat15['y']

fitted    = outcomes[-2][3].fit(X15, y15)
predicted = fitted.predict(X16)

In [20]:
pred = pd.DataFrame(predicted, columns=['predicted'])
dat16 = dat16.reset_index()
pred['y'] = dat16['y']

In [21]:
pred['flag'] = 0
pred['flag'][pred['predicted'].between(pred['y']*0.85, pred['y']*1.15)] = 1

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [22]:
pred.flag.sum()

344